# Handwritten Chinese and Japanese OCR

In this tutorial optical character recognition for handwritten Chinese (simplified) and Japanese is presented. Roman alphabet OCR can be find in [notebook 208](../208-optical-character-recognition). This model is capable of doing only one line of symbols each time. 

Models used for this notebooks are [handwritten-japanese-recognition](https://docs.openvinotoolkit.org/latest/omz_models_model_handwritten_japanese_recognition_0001.html) and [handwritten-simplified-chinese](https://docs.openvinotoolkit.org/latest/omz_models_model_handwritten_simplified_chinese_recognition_0001.html). To decode models output to readable text [kondate_nakayosi](https://github.com/openvinotoolkit/open_model_zoo/blob/master/data/dataset_classes/kondate_nakayosi.txt) and [scut_ept](https://github.com/openvinotoolkit/open_model_zoo/blob/master/data/dataset_classes/scut_ept.txt) charlists are used. Both model are from [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/).

## Imports modules required to run

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
from openvino.inference_engine import IECore
from os import path, makedirs

sys.path.append("../utils")
from notebook_utils import download_file

## Settings

In [ ]:
class Files:
    def __init__(self, model: str, charlist_link: str, charlist_name: str = None):
        self.model = model
        self.charlist_link = charlist_link
        self.charlist_name = charlist_name if charlist_name is not None else self.charlist_link.split('/')[-1]


ie = IECore()

model_folder = "model"
data_folder = "data"
charlist_folder = f"{model_folder}/charlists"

precision = "FP16"

model_extensions = ("bin", "xml")

handwritten_japanese_model_name = "handwritten-japanese-recognition-0001"
handwritten_simplified_chinese_model_name = "handwritten-simplified-chinese-recognition-0001"

japaneses_charlist_link = "https://raw.githubusercontent.com/openvinotoolkit/open_model_zoo/master/data/dataset_classes/kondate_nakayosi.txt"
simplified_chinese_charlist_link = "https://raw.githubusercontent.com/openvinotoolkit/open_model_zoo/master/data/dataset_classes/scut_ept.txt"

chinese_image_link = 'https://github.com/openvinotoolkit/open_model_zoo/raw/master/demos/handwritten_text_recognition_demo/python/data/handwritten_simplified_chinese_test.png'
japaneses_image_link = 'https://github.com/openvinotoolkit/open_model_zoo/raw/master/demos/handwritten_text_recognition_demo/python/data/handwritten_japanese_test.png'

japaneses_files = Files(handwritten_japanese_model_name, japaneses_charlist_link)
chinese_files = Files(handwritten_simplified_chinese_model_name, simplified_chinese_charlist_link)

## Download models and convert public model

If it is your first run models will download and convert here. It might take up to ten minutes. 

In [ ]:
makedirs(model_folder, exist_ok=True)

# Define which models to use, based on this line there will be downloaded nececery files used for OCR
used_files = (chinese_files, japaneses_files)

# Download models
for file_name in used_files:
    for extension in model_extensions:
        if not path.isfile(f'{model_folder}/intel/{file_name.model}/{precision}/{file_name.model}.{extension}'):
            download_command = f'omz_downloader --name {file_name.model} --output_dir {model_folder} --precision {precision}'
            ! $download_command

# Download charlists            
for file_name in used_files:
    if not path.isfile(f'{charlist_folder}/{file_name.charlist_name}'):
        download_file(file_name.charlist_link, directory=charlist_folder, show_progress=False)    

## Load the network

In [ ]:
# Choose which model to use

# Uncomment line below for Japaneses 
# currently_used_model = japaneses_files.model

# Uncomment line below for Chinese
currently_used_model = chinese_files.model

net = ie.read_network(
    model=f"{model_folder}/intel/{currently_used_model}/{precision}/{currently_used_model}.xml"
)

exec_net = ie.load_network(net, "CPU")

recognition_output_layer = next(iter(exec_net.outputs))
recognition_input_layer = next(iter(exec_net.input_info))

## Load an Image

In [ ]:
# Create data folder
makedirs(data_folder, exist_ok=True)

# Download image
image_link = chinese_image_link if currently_used_model == chinese_files.model else japaneses_image_link
file_name = image_link.split('/')[-1]
download_file(image_link, directory=data_folder, show_progress=False)

# Text detection models expects image in grayscale format
# IMPORTANT!!! This model allows to read only one line at time

# Read image
image = cv2.imread(f"{data_folder}/{file_name}", cv2.IMREAD_GRAYSCALE)

# Fetch shape
image_height, image_width = image.shape

# Calculate aspect ratio to keep text readable
aspect_ratio = image_width / image_height

# B,C,H,W = batch size, number of channels, height, width
_, _, H, W = net.input_info[recognition_input_layer].input_data.shape

# Resize image to meet network expected input sizes
resized_image = cv2.resize(image, (int(H * aspect_ratio), H), interpolation=cv2.INTER_AREA)

# Pad image to meet input size
resized_image = np.pad(resized_image, ((0, 0), (0, W - int(H * aspect_ratio))), mode='edge')

# Reshape to network input shape
input_image = resized_image[None, None, :, :]

plt.figure(figsize=(20, 1))
plt.axis('off')
plt.imshow(resized_image, cmap='gray', vmin=0, vmax=255);

## Prepare charlist

In [ ]:
# Get dictionary to encode output, based on model documentation
used_charlist = chinese_files.charlist_name if currently_used_model == handwritten_simplified_chinese_model_name else japaneses_files.charlist_name

# With both models, there should be blank symbol added at index 0 of each charlists
blank_char = '~'

with open(f"{charlist_folder}/{used_charlist}", 'r', encoding='utf-8') as charlist:
    letters = blank_char + ''.join(line.strip() for line in charlist)

## Run inference

In [ ]:
# Run inference on model
predictions = exec_net.infer(inputs={recognition_input_layer: input_image})[recognition_output_layer]

## Process infered data

In [ ]:
def threshold(prob, cnt, mult):
    return (prob / cnt) * mult


# Remove unnececery dimension
predictions = np.squeeze(predictions)

# Run argsort to pick two most possible symbols
predictions_indexes = np.argmax(predictions, axis=1)

# Define counter and probability to calculate avg probability difference between predicted symbol and blank symbol. 
cnt, prob = 0, 0
for i, p in enumerate(predictions_indexes):
    if p != 0:
        cnt += 1
        prob += predictions[i][p] - predictions[i][0]

output_text = []

## Print output

In [ ]:
# Create list of letters that difference between its calculated probability and probability of blank symbol is greater than given threshold. 
# WARNING! Threshold multiplier might be different for different models!
# To change output (e.g. if letters are missing, or duplicates are visible) modify threshold_multiplier

threshold_multiplier = 1 / 2

for i, p in enumerate(predictions_indexes):
    if p != 0:
        if predictions[i][p] - predictions[i][0] > threshold(prob, cnt, threshold_multiplier):
            output_text.append(letters[p])
            
plt.figure(figsize=(20, 1))
plt.axis('off')
plt.imshow(resized_image, cmap='gray', vmin=0, vmax=255)
print(''.join(output_text))
